# Welcome to the autopy API showcase

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:55% !important; }</style>"))

## Dependencies 

Autopy requires pyatoms, a job management package for atomistic calculations. Pyatoms handles the fundamental infrastructure for running DFT jobs via ASE. It supports calculators such as VASP/CP2K.

In [ ]:
from autopy.workflows.manager import Project
from pyatoms.jobs.job import OverallJobRunner # for real runs
from pyatoms.jobs.vasp.runner import FakeVaspOverallJobRunner # for fast testing

## Creating a project

The Project class is the main entry point. We first create a temporary directory for our project; you can choose any directory you like. The calculations and database will be found in this directory. 

The `Project` class is the main API for autopy.

In [ ]:
import os
from tempfile import TemporaryDirectory
tmpdir = TemporaryDirectory()
project_folder = os.path.join(tmpdir.name, 'test_Cu6')
os.mkdir(project_folder)

In [ ]:
print(project_folder)

We specify a "jobrunner" that contains all information about the server and how to run VASP jobs. For the sake of speed, here we use a "fake" jobrunner that will always give 123 as the energy.

In [ ]:
#jobrunner = OverallJobRunner(server='stratus', num_nodes=1) # use this for a real DFT run
jobrunner = FakeVaspOverallJobRunner() # use this for testing

The project name should be the same as that specified for the Project settings yaml file for pyatoms (in ~/.pyatoms/vasp/{project_name}.yaml) 

In [ ]:
project = Project(jobrunner=jobrunner, name='pbed3', folder=project_folder)

## Querying project for H adsorption on Cu(111)

Here is a simple example of a query. Only 3 attributes are required to minimally specify the query. Additional attributes are also available to further finetune the query.

In [ ]:
from autopy.database.ase_db import QuerySet
qs = QuerySet()
 
#NOTE: QuerySet attributes are defined by lists. 
# Multiple specifications can be specified per attribute. More shown in the later examples
# i.e., qs.chemsys = ['Cu', 'Rh']
# i.e., qs.miller_index = [(1,1,1), (1,0,0)]

qs.chemsys = ['Cu'] # list of chemical systems
qs.miller_index = [(1,1,1)] # list of miller indices
qs.adsorbate = ['H'] # list of adsorbates

qs.e_above_hull = ['<0.001'] # optional but highly recommended to reduce the number of bulk obtained from MP
qs.rotation = [0] # optional but highly recommended for asymmetrical adsorbates
qs.unitcell_size = [(2,2)]  # optional default = [(1,1)]

In [ ]:
# Full list of attributes
'''
# Bulk properties
bulk: [str]
e_above_hull: [str/float] #str must be Comparator
bulk_formula: [str]
chemsys: [str]
spacegroup: [int]

# Surface properties
termination: [str]
overlayer: [str]
miller_index: [tuple/list]
unitcell_size: [tuple/list]
num_layers: int = [int]
num_fixed_layers: [int]
vacuum: [int]
conventional: [bool]

# Adsorbate properties
adsorbate: [str]
adsorbate_formula: [str]
bonds: [[int]] #e.g. [[0]] for monodentate and [[0,1]] for bidentate
connectivity: [[int]] #e.g. [[0]]
avg_coord_num: [[int/float/str]] #e.g. [[7]] for monodentate and [[7,7]] for bidentate; str must be Comparator
rotation: [[int/float]] typing.Union[int, float] = None

NOTE:
Comparator string format: '>value', '<value', '>=value' or '<=value'
'''

We feed the query to the `retrieve()` method. If the database does not have the relevant information, calculations will be automatically assembled and performed to obtain the desired information.

Since this is a new project, there is no data available and  we have to wait a while for the calculations to finish. 

In [ ]:
results =  project.retrieve(query_set=qs)
print(f'Retrieved {len(results)} results.')

energies = results.energies()
print(f'Energies: {energies}')

'''
NOTE: To access other databse attributes from results...
    attr = [entry.attributes.attr for entry in results.entries]

Example:
    termination = [entry.attributes.termination for entry in results.entries]
'''

There are 4 energies, corresponding to adsorption at the 4 high symmetry sites, fcc, top, bridge, and hcp. Let's run the get_energy() method again. Since we have now done the calculations, this is much faster, as we just retrieve data from the database.

In [ ]:
results_again =  project.retrieve(query_set=qs)
print(f'Retrieved {len(results_again)} results.')

We can obtain the optimized structures from the `structures()` method of the `results`. Visualization of the structures reassures us that our results are correct.

In [ ]:
structures = results.structures()
print(f'Found {len(structures)} structures')

In [ ]:
from ase.visualize import view
view(structures)

Looking into the database, we can see 1 bulk calculation, 1 miller index (slab) calculation, and 4 adsorbate calculations.

In [ ]:
!cd $project_folder/database; ase db -c ++ project.db  relaxed=True

## Querying project for CO adsorption on Cu(111)

We can easily probe another adsorbate by changing the adsorbate name.



In [ ]:
qs = QuerySet()
qs.chemsys = ['Cu']
qs.miller_index = [(1,1,1)]
qs.adsorbate = ['CO']

results_co =  project.retrieve(qs)
print(f'Retrieved {len(results_co)} results')


structures_co = results_co.structures()
print(f'Found {len(structures_co)} structures')
view(structures_co)

Looking into the database again, we now see 1 bulk calculation, 1 miller index (slab) calculation, and 8 adsorbate calculations. We were able to reuse the bulk calculation and miller index calculation. Only the new adsorbate calculations were needed.

In [ ]:
!cd $project_folder/database; ase db -c ++ project.db  relaxed=True

NOTE: Sometimes autopy cannot automatically generate the adsorbate structure just from its name. In that case, you need to provide your own adsorbate structure file in ~/.autopy/adsorbates. The structure file has to be named as {adsorbate_name}.{any readable

## Querying multiple systems simultaneously

For efficiency, we can also query multiple systems by adding on to the list of specified attributes. This would be faster than querying systems individually. Here we get the results for adsorption of both CO and H.

In [ ]:
qs = QuerySet()
qs.chemsys = ['Cu']
qs.miller_index = [(1,1,1)]
qs.adsorbate = ['CO','H']
qs.unitcell_size = [(2,2)]  

results_multi =  project.retrieve(qs)
print(f'Retrieved {len(results_co)} results')


For further analysis, we can filter the results to pinpoint the results that we desire

In [ ]:
results_CO_111 = results_multi.filter(adsorbate='CO', miller_index=(1,1,1))
results_H_111 = results_multi.filter(adsorbate='H', miller_index=(1,1,1))
print(f'Retrieved {len(results_CO_111)} results for CO') # 4 sites for CO
print(f'Retrieved {len(results_H_111)} results for H') # 4 sites for H